In [1]:
import multiprocessing as mp
from string import ascii_uppercase
import random
from sys import stdout

**OBJETIVO:**

El objetivo de esta tarea consiste en poner en práctica los conceptos de cómputo concurrente vistos en clase. Se utilizarán procesos por medio del módulo `multiprocessing` del lenguaje de programación Python.

## Programa 1

1. Realiza el **Programa1** que instancie, como clase o con el método `Process`, 10 procesos. Cada uno de los procesos hijos recibirán un valor entero, y un caracter $(i,c)$ enviados por el proceso padre, los procesos escribirán en la salida estándar $i$ veces el caracter $c$.

In [2]:
def imprimir_mensaje(i, c):
    for _ in range(i):
        stdout.write(c)

In [3]:
num_procesos = 10
for i in range(num_procesos):
    c = ascii_uppercase[i]
    it = random.randint(3, 6)
    proceso = mp.Process(target=imprimir_mensaje, args=(it, c))
    proceso.start()

AACCACDDCBDBIDIEBDEBBBIGIFGJGEGFHHIIJFHJEFHH

## Programa 2

2. Refactoriza (reescribe) el programa anterior y elabora el **Programa2** que incluya un mecanismo de sincronización el cual permita escribir en orden todos los caracteres de cada proceso. Es decir, que se obtenga la secuencia $c_{1,1},\ldots,c_{i,1},c_{2,1},\ldots,c_{2,i},\ldots,c_{10,1},\ldots,c_{10,i}$, donde cada subsecuencia $c_{k,i}$ para cada $k=1,2,\ldots,10$ es la secuencia de caracteres del proceso hijo $k$ con longitud $i$.

In [4]:
mutex = mp.Lock()

In [5]:
def imprimir_mensaje(i, c):
    mutex.acquire()
    for _ in range(i):
        stdout.write(c)
    mutex.release()

In [6]:
num_procesos = 10
for i in range(num_procesos):
    c = ascii_uppercase[i]
    it = random.randint(3, 6)
    proceso = mp.Process(target=imprimir_mensaje, args=(it, c))
    proceso.start()

AAABBBBBBCCCCCDDDEEEEEEFFFFGGGGHHHHIIIIIIJJJJJJ

## Programa 3

Refactoriza (reescribe) el Programa2 y elabora el **Programa3** donde construyas un mecanismo de sincronización el cual permita escribir en orden todos los caracteres de cada proceso siguiendo una política de orden $p$ que será una lista de números enteros aleatorios con los índices $k$ de cada proceso hijo. La esccritura de los caracteres seguirá la secuencia de $c_{k,i}(p)$ donde cada secuencia $c_{k,i}$ estará definida por la política $p$.

In [7]:
def imprimir_mensaje(i, c, turno):
    turn = q.get()
    while True:
        mutex.acquire()
        if (turno.value == turn): # Si es el turno adecuado
            stdout.write("El turno de " +  str(c) + " es " + str(turn) + ". Se imprimirá " + str(i) + " veces\n")
            for _ in range(i):
                stdout.write(c)
            stdout.write("\n")
            turno.value = turno.value + 1
            mutex.release()
            break
        mutex.release()

In [8]:
mutex = mp.Lock()

# Politica de orden
p = list(range(10))
random.shuffle(p)
print('La politica de orden es: ', p)

q = mp.Queue()
for num in p:
    q.put(num)

num_procesos = 10
turno = mp.Value('i', 0) # Definimos un valor entero para llevar los turnos
procesos = [None]*num_procesos

for i in range(num_procesos):
    c = ascii_uppercase[i]
    it = random.randint(3, 6)
    proceso = mp.Process(target=imprimir_mensaje, args=(it, c, turno))
    procesos[i] = proceso

for p in procesos:
    p.start()

La politica de orden es:  [1, 0, 6, 2, 4, 3, 7, 5, 9, 8]
El turno de B es 0. Se imprimirá 3 veces
BBB
El turno de A es 1. Se imprimirá 3 veces
AAA
El turno de E es 2. Se imprimirá 3 veces
EEE
El turno de G es 3. Se imprimirá 3 veces
GGG
El turno de C es 4. Se imprimirá 5 veces
CCCCC
El turno de I es 5. Se imprimirá 3 veces
III
El turno de D es 6. Se imprimirá 3 veces
DDD
El turno de F es 7. Se imprimirá 6 veces
FFFFFF
El turno de J es 8. Se imprimirá 6 veces
JJJJJJ
El turno de H es 9. Se imprimirá 4 veces
HHHH
